In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [16]:
df= pd.read_csv('test.csv')
df_test = pd.DataFrame(df)

df= pd.read_csv('train.csv')
df_train = pd.DataFrame(df)



In [22]:
# dropping duplicate rows: PS: we dont have any 
df_train.drop_duplicates(inplace=True)
y_train = df_train['TARGET'].copy()
df_train_x = df_train.drop(columns="TARGET")



# df_train

In [24]:
print("number of rows", len(df_train))
print("number of colomns", len (df_train_x.T))

number of rows 76020
number of colomns 370


In [25]:
# dropping duplicated columns PS: there are no duplicate columns
df_train_x.T.drop_duplicates(inplace=True)

In [31]:
# dropping columns with variance 0 PS: drops 34 columns
df_train_x = df_train_x.drop(df_train_x.var()[df.var() == 0].index.values, axis=1)

In [41]:
# dropping columns starting with ind_* they cover the same information
column_to_drop = [name for name in df_train_x if name.startswith("ind_")]
df_train_x = df_train_x.drop(columns = column_to_drop )